# Problem 1: Gamma MLE Simulation Study

First, we define the negative loglikelihood function give the gamma parameters `theta` and data vector `x`.

In [1]:
import numpy as np
from scipy.stats import gamma
from scipy.optimize import minimize

def nllk_gamma(theta, x):
    return(- np.sum(gamma.logpdf(x, a = theta[0], scale = theta[1])))

Let's test it out.

In [2]:
n = 100
theta = [2., 2.]
x = gamma.rvs(a = theta[0], scale = theta[1], size = n)

nllk_gamma(theta, x)

231.47722169902787

Now let's write a function to find the MLE by minimizing the negative loglikelihood function.

In [3]:
def mle_gamma(x):
    ## moment estimator as initial value
    m = x.mean()
    v = x.var()
    init = np.array([m * m / v, v / m])
    ## optimize
    fit = minimize(
        nllk_gamma,
        x0  = init,
        args = (x),
        method = "BFGS")
    mle = fit.x                         # MLE
    se = np.sqrt(np.diag(fit.hess_inv)) # standard error
    return mle, se

Let's see if this works.

In [4]:
mle, se = mle_gamma(x)

In [5]:
mle

array([1.72977825, 2.30939004])

In [6]:
se

array([0.22527359, 0.34875206])

Now we can design a simulation study with number of replications `nrep`, true parameter vector `theta`, and sample size `n`.

In [7]:
def mysim(nrep, theta, n):
    mle = np.empty((nrep, len(theta)))
    se  = np.empty((nrep, len(theta)))
    for i in range(nrep):
        x = gamma.rvs(a = theta[0], scale = theta[1], size = n)
        mle[i,], se[i,] = mle_gamma(x)
    return {'mle': mle, 'se': se}

It works as expected:

In [8]:
sim = mysim(5, theta, 100)

In [9]:
sim

{'mle': array([[2.88430936, 1.36793522],
        [2.27448023, 1.68518255],
        [2.31764838, 1.69055701],
        [2.39758087, 1.51398333],
        [1.88733887, 1.79588691]]),
 'se': array([[0.38678602, 0.20035692],
        [0.3007394 , 0.24994273],
        [0.30896184, 0.25151033],
        [0.31828873, 0.22364237],
        [0.24738054, 0.26859417]])}

To summarize the results of a simulation study, we investigate: 1) whether the mean of the estimates match the true parameters; and 2) whether the mean of the standard errors match the empirical standard errors.

In [10]:
def sumsim(sim):
    avg = sim['mle'].mean(axis = 0)
    ase = sim['se'].mean(axis = 0)
    ese = sim['mle'].std(axis = 0)
    return [avg, ase, ese]

In [11]:
sumsim(sim)

[array([2.35227154, 1.610709  ]),
 array([0.31243131, 0.23880931]),
 array([0.31891372, 0.15133264])]

We are ready to perform a few simulation studies and summarize the results.

In [12]:
nrep = 1000
sim_50 = mysim(nrep, theta, 50)
sumsim(sim_50)

[array([2.09082776, 1.97218963]),
 array([0.39001695, 0.41689654]),
 array([0.41083031, 0.42194625])]

In [13]:
sim_100 = mysim(nrep, theta, 100)
sumsim(sim_100)

[array([2.04792242, 1.9938062 ]),
 array([0.26970189, 0.29771504]),
 array([0.27746879, 0.30532247])]

In [14]:
sim_200 = mysim(nrep, theta, 200)
sumsim(sim_200)

[array([2.03408097, 1.98374012]),
 array([0.18955294, 0.20911948]),
 array([0.19560285, 0.21151545])]

The results suggest that the MLEs recover the true parameter values and that the standard errors of the MLEs mathch their empirical standard errors. The agreement improves as sample size increases.

__The title and author field did not show. I haven't figured this out. Some help would be appreciated.__